In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.batchnorm1 = nn.BatchNorm2d(8)
        self.dropout1 = nn.Dropout(0.1)

        self.conv2 = nn.Conv2d(8, 16, 3,padding = 1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.batchnorm2 = nn.BatchNorm2d(16)

        self.conv3 = nn.Conv2d(16, 32, 3)

        self.fc1 = nn.Linear(800,10)

    def forward(self, x):
        # print(x.shape)
        x = F.relu(self.conv1(x))
        # print(x.shape)
        x = self.dropout1(self.batchnorm1(self.pool1(x)))
        # print(x.shape)
        x = F.relu(self.conv2(x))
        # print(x.shape)
        x = self.batchnorm2(self.pool2(x))
        # print(x.numel(), x.shape, x.shape[0])
        x = F.relu(self.conv3(x))

        # print(x.numel(), x.shape, x.shape[0])

        # x = x.view( -1, x.numel())
        # print(x.shape)

        x = x.view(x.shape[0], -1)
        # print(x.shape)
        x = self.fc1(x)

        return F.log_softmax(x)

In [4]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
         MaxPool2d-2            [-1, 8, 14, 14]               0
       BatchNorm2d-3            [-1, 8, 14, 14]              16
           Dropout-4            [-1, 8, 14, 14]               0
            Conv2d-5           [-1, 16, 14, 14]           1,168
         MaxPool2d-6             [-1, 16, 7, 7]               0
       BatchNorm2d-7             [-1, 16, 7, 7]              32
            Conv2d-8             [-1, 32, 5, 5]           4,640
            Linear-9                   [-1, 10]           8,010
Total params: 13,946
Trainable params: 13,946
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.13
Params size (MB): 0.05
Estimated Total Size (MB): 0.18
---------------------------------------------

<ipython-input-3-28bcb0371ba1>:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [8]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [6]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # print("data shape ", data.shape)
        # print("output shape ", output.shape)

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'Train: Loss={loss.item():0.4f} Batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]<ipython-input-3-28bcb0371ba1>:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Train: Loss=0.1346 Batch_id=937: 100%|██████████| 938/938 [00:39<00:00, 23.46it/s]



Test set: Average loss: 0.0474, Accuracy: 9839/10000 (98.39%)



Train: Loss=0.0041 Batch_id=937: 100%|██████████| 938/938 [00:39<00:00, 24.00it/s]



Test set: Average loss: 0.0397, Accuracy: 9865/10000 (98.65%)



Train: Loss=0.0009 Batch_id=937: 100%|██████████| 938/938 [00:40<00:00, 23.38it/s]


...


Test set: Average loss: 0.0233, Accuracy: 9929/10000 (99.29%)



Train: Loss=0.0001 Batch_id=937: 100%|██████████| 938/938 [00:39<00:00, 23.98it/s]



Test set: Average loss: 0.0266, Accuracy: 9914/10000 (99.14%)



Train: Loss=0.0000 Batch_id=937: 100%|██████████| 938/938 [00:39<00:00, 23.81it/s]



Test set: Average loss: 0.0276, Accuracy: 9919/10000 (99.19%)

